### 1. 패키지 설치

In [ ]:
pip install --upgrade pip

In [1]:
pip install -r requirements.txt  # 패키지 설치

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reason for being yanked: <none given>
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 38.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 44.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 47.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
print(torch.__version__)  # PyTorch 버전
print(torch.version.cuda)  # PyTorch가 사용하는 CUDA 버전
print(torch.cuda.is_available())  # GPU 사용 가능 여부


2.4.1+cu121
12.1
True


### 2. 문서 split 및 Chroma를 활용한 vector store 구성

In [16]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"로그인 상태입니다. 사용자: {user_info['name']}")
except Exception as e:
    print("로그인되지 않았거나 토큰이 유효하지 않습니다.")
    print(e)

로그인 상태입니다. 사용자: chaeeee


In [17]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "false"

In [ ]:
from langchain.schema import Document
import os, json
from tqdm import tqdm

# 1. JSON 파일 경로 설정
json_path = "./json_data.json"  # 단일 JSON 파일 경로

# 2. JSON 데이터 불러오기
with open(json_path, 'r', encoding='utf-8') as f:
    all_data = json.load(f)

# 3. {}를 기준으로 JSON 데이터 분할 및 Document 객체 생성
documents = []

for data in tqdm(all_data, desc="Generating Documents", unit="entry", ncols=80):
    metadata = {
        "title": data.get('title', 'N/A'),
        "artist": data.get('artist', 'N/A'),
        "year": data.get('year', 'N/A'),
        "read_count": data.get('read_count', 0)
    }

    # JSON 데이터의 각 항목을 Document 객체로 변환
    doc_content = json.dumps(data, ensure_ascii=False, indent=4)
    documents.append(Document(
        page_content=doc_content.strip(),
        metadata=metadata
    ))

In [ ]:
len(documents)

In [ ]:
documents[1]

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.schema import Document

# 텍스트 분할 설정
text_splitter = CharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=128  # 오버랩 설정
)

# 단일 DOCX 파일 로드
file_path = "./dataset2.docx"  # 파일 경로를 이곳에 입력하세요
loader = Docx2txtLoader(file_path)
raw_text = loader.load()[0].page_content  # DOCX 파일의 전체 텍스트 가져오기
print("raw_text 개수: ", len(raw_text))
      
# 작품명을 기준으로 텍스트 분리
def split_artwork_documents(doc_text):
    artworks = doc_text.split("\n\n작품명:")  # 작품을 구분
    documents = []

    for artwork in artworks:
        if artwork.strip():  # 빈 텍스트 제외
            # "작품명:" 추가로 일관성 유지
            doc_content = "작품명:" + artwork if not artwork.startswith("작품명:") else artwork

            # 메타데이터 초기화
            metadata = {}
            lines = doc_content.split("\n")  # 텍스트 줄 단위로 나누기

            # 메타데이터 추출
            for line in lines:
                if line.startswith("작품명:"):
                    metadata["작품명"] = line.replace("작품명:", "").strip()
                elif line.startswith("작가:"):
                    metadata["작가"] = line.replace("작가:", "").strip()
                elif line.startswith("제작 연도:"):
                    metadata["제작 연도"] = line.replace("제작 연도:", "").strip()
                elif line.startswith("카테고리:"):
                    metadata["카테고리"] = line.replace("카테고리:", "").strip()

            # Document 객체 생성
            documents.append(Document(
                page_content=doc_content.strip(),
                metadata=metadata
            ))

    return documents



# 작품별 Document 생성
documents = split_artwork_documents(raw_text)

# 생성된 작품별 Document에 대해 추가 청크 분할
chunked_documents = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)
    for chunk in chunks:
        # 청크에 원래 Document의 메타데이터 유지
        chunked_documents.append(Document(
            page_content=chunk,
            metadata=doc.metadata  # 원본 메타데이터 복사
        ))

print(f"총 문서 수: {len(chunked_documents)}")

In [ ]:
documents[11001]

In [ ]:
# 특정 Document 객체의 텍스트 길이 확인
len(documents[11001].page_content)

In [ ]:
# 800자를 초과하는 Document 개수 세기
over_800_count = sum(1 for doc in documents if len(doc.page_content) > 800)

# 결과 출력
print(f"800자를 초과하는 Document 개수: {over_800_count}")


In [ ]:
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import faiss  # FAISS 라이브러리 필요

# 1. 임베딩 초기화
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


In [ ]:
# 2. 문서 데이터와 메타데이터 분리
texts = [doc.page_content for doc in documents]  # 문서 텍스트
metadatas = [doc.metadata for doc in documents]  # 문서 메타데이터

# 3. 문서 임베딩 생성
embeddings = embedding_model.encode(texts)

In [ ]:
# 4. FAISS 인덱스 생성
embedding_dim = embeddings.shape[1]  # 벡터 차원 확인
faiss_index = faiss.IndexFlatL2(embedding_dim)  # L2 거리 기반 인덱스
faiss_index.add(embeddings)  # 벡터 추가


In [ ]:
from langchain.docstore.in_memory import InMemoryDocstore

# 5. Docstore 생성
# 각 문서에 고유 ID를 부여해 InMemoryDocstore 생성
docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})

In [ ]:
# 6. FAISS 벡터스토어 생성
def embed_query(text):
    return embedding_model.encode([text])[0]  # 단일 쿼리 텍스트를 임베딩

index_to_docstore_id = {i: str(i) for i in range(len(documents))}

faiss_db = FAISS(
    embedding_function=embed_query,
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# 7. FAISS 데이터베이스 저장
faiss_db.save_local("./faiss_artworks_0114_docx")
print("FAISS 데이터베이스가 성공적으로 저장되었습니다!")


In [ ]:
# 9. 검색 테스트
query = "박승무의 설경"
results = faiss_db.similarity_search(query, k=5)

# 10. 검색 결과 출력
for result in results:
    print("문서 텍스트:", result.page_content)
    print("문서 메타데이터:", result.metadata)
    print('=------------------------------------------')


In [1]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


/home/chae/faiss_env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
You try to use a model that was created with version 3.3.1, however, your version is 3.2.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [84]:
from langchain_community.vectorstores import FAISS


# 기존 DB 로드 
persist_directory = "./faiss_artworks_0114"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 로드되었습니다!


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4bit 양자화 활성화
    bnb_4bit_compute_dtype=torch.float16,  # 계산 타입 설정 (float16이 일반적)
    bnb_4bit_use_double_quant=True,  # 더블 양자화 사용 (메모리 절약)
    bnb_4bit_quant_type="nf4",  # NormalFloat4 (NF4) 사용
)

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit 양자화 활성화
    bnb_4bit_compute_dtype="float16",  # 계산 정밀도 설정
    bnb_4bit_quant_type="nf4",  # NF4 양자화 방식 사용 (효율적)
    bnb_4bit_use_double_quant=True,  # 이중 양자화 사용
)

In [5]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,  # ✅ 올바른 양자화 설정 적용
    device_map="auto",  # ✅ 자동 GPU 배치
    trust_remote_code=True,
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:27<00:00,  6.98s/it]


In [6]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipykernel_301791/1315975019.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [99]:
from langchain.prompts import ChatPromptTemplate

deepseek_template = """
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions **accurately based on the provided document (context)**. 
If the requested information is not found in the document, respond with:
"문서에 해당 정보가 없습니다." 

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in Korean.
Use **beautiful Markdown formatting** (headings, bullet points, **bold** or *italic* text) to enhance readability.
You must include the artwork number in your response.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
"""




exaone_template = '''
<|system|>
You are an AI assistant tasked with refining and polishing the provided logical reasoning into a final answer in Korean.  
Your role is to produce a clear, concise, and well-structured response that maintains the original meaning and key details.  
Ensure that your final answer is written in Korean and uses **beautiful Markdown formatting** (e.g., headings, bullet points, **bold** or *italic* text) to enhance readability.  
Focus solely on refining the content without adding any new information.

<|reasoning|>
{reasoning}

<|user|>
Based on the above reasoning, please generate a refined and final answer in Korean.

<|assistant|>
'''



# DeepSeek 템플릿 생성
deepseek_prompt = ChatPromptTemplate.from_template(deepseek_template)

# EXAONE 템플릿 생성
exaone_prompt = ChatPromptTemplate.from_template(exaone_template)



In [86]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with:
"The document does not contain this information." 

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in Korean.
Use **beautiful Markdown formatting** (headings, bullet points, bold or italic text) to enhance readability.
You must include artwork number.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [89]:
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 15,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.3      # 다양성과 관련성 간 균형
    }
)


In [90]:
import re

class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"</think>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [109]:
import re

class MarkdownOutputParser2:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [110]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# 🔹 EXAONE 모델 로드
exaone_model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
exaone_tokenizer = AutoTokenizer.from_pretrained(exaone_model_id)
exaone_model = AutoModelForCausalLM.from_pretrained(
    exaone_model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:25<00:00,  3.63s/it]


In [111]:
from transformers import pipeline

# 파이프라인 생성
exaone_pipe = pipeline(
    "text-generation",
    model=exaone_model,
    tokenizer=exaone_tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
exaone_llm = HuggingFacePipeline(pipeline=exaone_pipe)

In [112]:
from langchain.schema.runnable import RunnableLambda

chain = (
    retriever
    | RunnableLambda(lambda docs: {  
        "context": "\n".join([doc.page_content for doc in docs]),  
        "question": query 
    })
    | deepseek_prompt
    | llm
    | MarkdownOutputParser()
    | (lambda x: {"reasoning": x})
    | exaone_prompt
    | exaone_llm
    | MarkdownOutputParser2()
)


In [115]:
query = "2000년대 유화작품 2개 추천해줘?"

In [116]:
response = chain.invoke({"question": query})
print(response)

### **🔹 모델 응답 결과**

## 🖼️ 2000년대 유화 작품 추천

2000년대의 주목할 만한 유화 작품 두 점을 소개합니다:

* **《아이리스》 (6609)**
    * **작가**: 김점선
    * **연도**: 2001
    * **크기**: 199.5 x 139.3 cm
    * **재료**: 캔버스에 유화 물감
    * **특징**: 김점선 작가는 독창적인 시각으로 작품 세계를 구축했습니다. 《아이리스》는 화려한 색감과 역동적인 붓터치가 생명력 넘치는 분위기를 자아냅니다.

* **《정물화 2》 (4483)**
    * **작가**: 김지원
    * **연도**: 2000
    * **크기**: 194 x 130 cm
    * **재료**: 캔버스에 유화 물감
    * **특징**: 김지원 작가는 일상적인 사물들을 통해 새로운 해석을 제시합니다. 《정물화 2》는 사실적인 표현 속에 경쾌한 붓터치가 더해져 사물의 독창성과 생동감을 잘 드러냅니다.

두 작품 모두 2000년대 유화 예술의 특징을 잘 보여주는 대표작입니다.


In [96]:
retrieved_docs = retriever.get_relevant_documents(query)
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")  # 문서의 실제 내용
    print(f"Metadata: {doc.metadata}")    # 메타데이터 (예: 출처, 페이지 등)
    print("-" * 50)


Document 1:
Content: {
    "title": "이호섭 인물",
    "title_ch": "李湖燮 人物",
    "title_eng": "Portrait of Lee, Ho-Sup",
    "artist": "임응식",
    "artist_eng": "LIMB Eungsik",
    "artwork_number": 663,
    "year": "1982",
    "size": "33×25.7",
    "materials": "종이에 젤라틴실버프린트",
    "category": "사진",
    "description": "한국 사진계의 선구자인 임응식(1912-2001)은 사진의 기록성, 현실성 등 사진매체의 본질에 주목하여 인간 생활사를 표현하였던 작가이다. 그는 1950년대 이후 ‘생활주의 사진운동’을 일으켜 리얼리즘(Realism)에 입각한 사진을 제작하였으며, 자신의 사진관에 대해 \"사진이란 인간생활의 기록이고 진실이다\"라고 하였다.임응식은 노년기에 들어서면서부터 친분이 있는 주위의 예술가들을 주제로 인물 사진을 찍었다. 사실주의에 입각하여 우리의 생활과 역사를 기록해오던 그가 유명 예술인의 모습을 촬영한 이유는, 늦기 전에 그들의 가치를 기록해 두어야 한다는 사명감 때문이었다. 그는 자신의 마지막 작업으로써 동료 예술인들을 필름에 담는 것은 자신의 ‘임무’라고 사진집『풍모』(1982)의 서문에서 밝힌 바 있다.이러한 작업의 일환으로 제작된 <이호섭 인물>(1982)은 작곡가 이호섭(李湖燮)을 촬영한 사진이다. 이호섭의 대표적인 가곡으로는 <그리움>, <기다림> 등이 있다.",
    "read_count": 4
}
Metadata: {'title': '이호섭 인물', 'artist': '임응식', 'year': '1982', 'read_count': 4}
--------------------------------------------------
Document 2:
Content: {
    "title": "

In [55]:
# 검색 수행: 유사도 점수와 함께 반환
docs_and_scores = retriever.vectorstore.similarity_search_with_score(query, k=5)

# 검색된 문서 수 출력
print(f"검색된 문서 수: {len(docs_and_scores)}")

# 각 문서의 파일명, 전체 내용, 유사도 점수 출력
for i, (doc, score) in enumerate(docs_and_scores, 1):
    print(f"\n문서 {i}:")
    print(f"  파일명: {doc.metadata.get('source', 'N/A')}")
    print(f"  유사도 점수: {score:.4f}")
    print(f"  전체 내용: {doc.page_content}")


검색된 문서 수: 5

문서 1:
  파일명: N/A
  유사도 점수: 0.4776
  전체 내용: 작품명: 토끼풀 / N/A / Clover

작가: 이중섭 / LEE Jungseop

작품 번호: 10001

제작 연도: 1941

크기: 9×14

재료: 종이에 펜, 수채 물감

카테고리: 드로잉

작품 설명: 대향(大鄕) 이중섭(李仲燮, 1916-1956)은 평안북도 정주의 오산고등보통학교에서 서양화가 임용련, 백남순 부부에게 서양화를 배웠다. 이후 1936년 일본으로 건너가 데이코쿠미술학교(帝国美術学校)와 분카학원(文化学院)에서 미술을 전공했다. 추상 미술단체인 ‘자유미술가협회(自由美術家協会)’의 전시회에 지속적으로 출품하였으며, 제7회전(1943)에서는 태양상(太陽賞)을 수상했다. 1943년 귀국 후에는 생활고와 병으로 고생하면서도 꾸준히 작품을 제작했다.이중섭은 소, 아이들 등을 주요 소재로 고분 벽화와 민화 등 전통적이고 토속적인 것에 영감을 받아 표현주의적인 감각으로 작품을 제작했다. 이중섭의 작품에서는 그의 삶을 엿볼 수 있다. 동경의 분카학원에서 야마모토 마사코와 연애하던 시기의 엽서화에는 두 사람의 연인관계를 암시하는 환상적이고 초현실주의적인 이미지를 그렸다. 한국 전쟁기 제주도 피란시절 작품에는 가족과 행복했던 나날들이 소박하게 표현되었으며, 가족을 일본으로 보낸 후에는 삭막한 풍경화와 전쟁의 은유들이 그려졌다. 그는 열악한 경제 상황과 재료 부족에도 끊임없이 새로운 기법과 재료를 실험했는데, 담배를 싼 은지를 활용한 은지화가 대표적인 예이다. 전쟁이 끝난 후에는 가족을 만나려는 생각에 작품 제작에 몰두하여, 당당하고 힘찬 기세가 화면에 나타난다. 그러나 곧 경제적 어려움과 정신질환 등에 시달리며 가족과 재회할 수 있으리라는 희망이 사라졌을 때에는 초점을 잃은 흐릿한 풍경들이 애잔하게 펼쳐졌다.〈토끼풀〉은 이중섭이 이 시기 제작한 여러 점의 엽서화 중 하나로, 토끼풀 도상은 그의 그림에 반복적으로 등장하였다. 화면 중앙에 분리된 꽃과 잎